In [198]:
import pandas as pd
import re

csvfile = "oliebollentest.csv"

In [199]:
oliebollentest2014 = pd.read_csv('../2014/oliebollentest_2014.csv')
oliebollentest2015 = pd.read_csv('../2015/oliebollentest_2015.csv')
oliebollentest2016 = pd.read_csv('../2016/oliebollentest_2016.csv')
oliebollentest2017 = pd.read_csv('../2017/oliebollentest_2017.csv')

In [200]:
oliebollentest2014['Jaar'] = '2014'
oliebollentest2015['Jaar'] = '2015'
oliebollentest2016['Jaar'] = '2016'
oliebollentest2017['Jaar'] = '2017'

In [201]:
for df in [oliebollentest2014, oliebollentest2015, oliebollentest2016, oliebollentest2017]:
    print(df.columns)

Index(['Eindcijfer', 'Gemiddeld gewicht (gr)', 'Hoeveelheid vulling',
       'Klefheid', 'Luchtigheid', 'Naam', 'Plaats', 'Plaats op ranglijst',
       'Prijs per stuk', 'Smaak', 'Uiterlijk', 'Vetpercentage', 'Zoetheid',
       'Jaar'],
      dtype='object')
Index(['Aanbieding in €', 'Adres', 'Adres_google', 'Categorie', 'Eindcijfer',
       'Eindconclusie', 'Gewicht (gram)', 'Naam', 'Plaats',
       'Plaats op ranglijst', 'Prijs per stuk (euro)', 'Vetpercentage %',
       'Jaar'],
      dtype='object')
Index(['Aanbieding in €', 'Adres', 'Categorie', 'Conclusie', 'Eindcijfer',
       'Gewicht (gram)', 'Naam', 'Nevenvestiging', 'Opmerking', 'Plaats',
       'Plaats op ranglijst', 'Prijs per stuk in €', 'Vet in %', 'Jaar'],
      dtype='object')
Index(['Aanbieding', 'Adres', 'Conclusie', 'Gewicht in gram', 'Naam', 'Plaats',
       'Plaats op ranglijst', 'Prijs per stuk in euro', 'Vetpercentage',
       'Zelf aangemeld voor deelname', 'Jaar'],
      dtype='object')


In [202]:
oliebollentest2014.rename(columns = {
    'Gemiddeld gewicht (gr)':'Gewicht in gram',
    'Prijs per stuk': 'Prijs per stuk in €'
    }, inplace = True)
oliebollentest2015.rename(columns = {
    'Gewicht (gram)':'Gewicht in gram',
    'Vetpercentage %': 'Vetpercentage',
    'Prijs per stuk (euro)': 'Prijs per stuk in €',
    'Eindconclusie': 'Conclusie'
}, inplace = True)
oliebollentest2016.rename(columns = {
    'Gewicht (gram)':'Gewicht in gram',
    'Vet in %': 'Vetpercentage',
}, inplace = True)
oliebollentest2017.rename(columns = {
    'Aanbieding':'Aanbieding in €',
    'Prijs per stuk in euro': 'Prijs per stuk in €'
}, inplace = True)

In [203]:
entry_dataframe = oliebollentest2014.append(oliebollentest2015).append(oliebollentest2016).append(oliebollentest2017)
list(entry_dataframe)

['Aanbieding in €',
 'Adres',
 'Adres_google',
 'Categorie',
 'Conclusie',
 'Eindcijfer',
 'Gewicht in gram',
 'Hoeveelheid vulling',
 'Jaar',
 'Klefheid',
 'Luchtigheid',
 'Naam',
 'Nevenvestiging',
 'Opmerking',
 'Plaats',
 'Plaats op ranglijst',
 'Prijs per stuk in €',
 'Smaak',
 'Uiterlijk',
 'Vetpercentage',
 'Zelf aangemeld voor deelname',
 'Zoetheid']

In [204]:
def removedoublespaces(city):
    if city is not None:
        newcity = re.sub(' +',' ',city).strip()
    else:
        newcity = None
    if city != newcity:
        print(str(city)+" --> "+str(newcity))
    return newcity

columns = ['Naam', 'Plaats']
for column in columns:
    entry_dataframe[column] = entry_dataframe[column].apply(removedoublespaces)

Richard  Visser’s Gebakkraam --> Richard Visser’s Gebakkraam
Bakkerij  Van den Berg --> Bakkerij Van den Berg
Gebakkraam ’t  Krulletje jr --> Gebakkraam ’t Krulletje jr
Meesterbakker van Iersel Slimstraat  --> Meesterbakker van Iersel Slimstraat


In [205]:
def rename(name):
    lowercasewords = ['van', 'jr', 'of', 'en', 'and', 'den', 'der', 'in', 'de', 'dé', 'by']
    
    words = name.split(' ')
    changewords = words[:]
    i=0
    for word in words:
        if not word[0].isalpha():
            pass
        elif word[0].isupper():
            pass
#             if word.lower() in lowercasewords and not i==0:
#                 changewords[i] = word.lower()
        else:
            if word.lower() not in lowercasewords:
                changewords[i] = word.capitalize()
        i+=1
    
    if words != changewords:
        newname = ' '.join(changewords)
        print(str(name)+" --> "+str(newname))
        return newname
    else:
        return name

entry_dataframe['Naam'] = entry_dataframe['Naam'].apply(rename)

Gebakkraam familie Smit --> Gebakkraam Familie Smit
De bakkers Lamers --> De Bakkers Lamers
Herweijer kwaliteitsbakker --> Herweijer Kwaliteitsbakker
Gebakkraam R.H.C. Hendriks en zonen --> Gebakkraam R.H.C. Hendriks en Zonen
Gebakkraam familie Smit --> Gebakkraam Familie Smit
De bakkers Lamers --> De Bakkers Lamers
Bakkerij Bi’j oons --> Bakkerij Bi’j Oons
Mirella’s oliebollerie --> Mirella’s Oliebollerie
Dukers dé bakker --> Dukers dé Bakker
Irene’s ambachtelijke bakkerswinkel --> Irene’s Ambachtelijke Bakkerswinkel
Bakkerij Neplenbroek - Wc. De gaard --> Bakkerij Neplenbroek - Wc. De Gaard


In [206]:
def updatecolumncapitalisation(column, desiredvalue):
    applicablerows = entry_dataframe[column].str.lower() == desiredvalue.lower()
    entry_dataframe.loc[applicablerows, column] = desiredvalue

for name in ['Bakkerij Aad Klootwijk - In Dirk', 'Meesterbakker Van Iersel', 'Bakkerij Van de Mortel',
             'Bakkerij Van Dongen']:
    updatecolumncapitalisation('Naam', name)

print(entry_dataframe[entry_dataframe['Naam'].str.lower() == 'Bakkerij Aad Klootwijk - In Dirk'.lower()]['Naam'])
print(entry_dataframe[entry_dataframe['Naam'].str.lower() == 'Meesterbakker Van Iersel'.lower()]['Naam'])
print(entry_dataframe[entry_dataframe['Naam'].str.lower() == 'Bakkerij van de Mortel'.lower()]['Naam'])
print(entry_dataframe[entry_dataframe['Naam'].str.lower() == 'Bakkerij Van Dongen'.lower()]['Naam'])

175    Bakkerij Aad Klootwijk - In Dirk
177    Bakkerij Aad Klootwijk - In Dirk
180    Bakkerij Aad Klootwijk - In Dirk
182    Bakkerij Aad Klootwijk - In Dirk
185    Bakkerij Aad Klootwijk - In Dirk
189    Bakkerij Aad Klootwijk - In Dirk
196    Bakkerij Aad Klootwijk - In Dirk
Name: Naam, dtype: object
14    Meesterbakker Van Iersel
43    Meesterbakker Van Iersel
43    Meesterbakker Van Iersel
Name: Naam, dtype: object
21    Bakkerij Van de Mortel
24    Bakkerij Van de Mortel
Name: Naam, dtype: object
40    Bakkerij Van Dongen
25    Bakkerij Van Dongen
Name: Naam, dtype: object


In [207]:
# Dre's Bakery
# Dre’s Bakery

entry_dataframe['Naam'] = entry_dataframe['Naam'].str.replace('\’', '\'')

In [225]:
def updatecolumn(column, desiredvalue, listofvalues):
    for value in listofvalues:
        entry_dataframe[column] = entry_dataframe[column].replace(value, desiredvalue)

updatecolumn('Naam', 'De Koning, De Echte Bakker', ['Echte Bakker De Koning', 'Koning, de Echte Bakker'])
updatecolumn('Naam', 'Oud-Hollandse Gebakkraam Aart van de Weerdt', ['Oud Hollandse Gebakkraam Aart van de Weerdt'])
updatecolumn('Naam', 'Gebakkraam Van Reken & Zn', ['Gebakkraam Van Reken'])
updatecolumn('Naam', 'De Wateringse Oliebollenkraam', ['Wateringse Oliebollenkraam'])
updatecolumn('Naam', 'Bakker Van Keulen', ['Bakkerij Van Keulen'])

updatecolumn('Naam', 'Gebakkraam Van Reken & Zn', ['Gebakkraam Van Reken', 'Gebakkraam Van Reken & Zn'])
updatecolumn('Naam', 'Hollandse Gebakkraam J. Witstijn', ['Hollandse Gebakkraam J. Witstijn', 'Hollandse Gebakkraam J.Witstijn'])
updatecolumn('Naam', 'Het Suikertuintje van de Betuwe', ['Het Suikertuintje van de Betuwe', 'Suikertuintje van de Betuwe'])
updatecolumn('Naam', 'Hollandse Gebakkraam Huijbrechts', ['Hollandse Gebakkraam Huijbrechts', 'Hollandse Gebakkraam Huijbregts'])


In [209]:
entry_dataframe

,Aanbieding in €,Adres,Adres_google,Categorie,Conclusie,Eindcijfer,Gewicht in gram,Hoeveelheid vulling,Jaar,Klefheid,...,Nevenvestiging,Opmerking,Plaats,Plaats op ranglijst,Prijs per stuk in €,Smaak,Uiterlijk,Vetpercentage,Zelf aangemeld voor deelname,Zoetheid
0,NaN,NaN,NaN,NaN,NaN,10.0,93.0,53.7,2014,20.8,...,NaN,NaN,IJsselstein,1.0,0.94,63.6,82.6,13.3,NaN,45.7
1,NaN,NaN,NaN,NaN,NaN,9.5,101.0,41.2,2014,21.9,...,NaN,NaN,Rossum,2.0,0.85,62.9,81.6,12.2,NaN,42.2
2,NaN,NaN,NaN,NaN,NaN,9.5,90.0,55.3,2014,21.4,...,NaN,NaN,Rotterdam,3.0,1,64.0,81.3,13.4,NaN,46.3
3,NaN,NaN,NaN,NaN,NaN,9.5,110.0,47.7,2014,20.8,...,NaN,NaN,Maarssen,4.0,1,60.0,81.7,13.6,NaN,43.5
4,NaN,NaN,NaN,NaN,NaN,9.0,96.0,50.3,2014,22.2,...,NaN,NaN,Spijkenisse,5.0,1,56.8,81.8,15.6,NaN,41.2
5,NaN,NaN,NaN,NaN,NaN,9.0,92.0,51.0,2014,28.2,...,NaN,NaN,Zoetermeer,6.0,0.9,64.5,81.5,12.3,NaN,44.9
6,NaN,NaN,NaN,NaN,NaN,9.0,91.0,51.9,2014,17.1,...,NaN,NaN,Tilburg,7.0,1,60.9,81.8,12,NaN,41.5
7,NaN,NaN,NaN,NaN,NaN,8.5,95.0,51.0,2014,23.3,...,NaN,NaN,Papendrecht,8.0,1,60.7,79.4,12.1,NaN,43.5
8,NaN,NaN,NaN,NaN,NaN,8.5,93.0,37.3,2014,26.7,...,NaN,NaN,Kortenhoef,9.0,0.95,59.9,80.4,12.3,NaN,42.3
9,NaN,NaN,NaN,NaN,NaN,8.5,90.0,51.3,2014,32.0,...,NaN,NaN,Schipluiden,10.0,0.9,63.5,75.6,10.9,NaN,46.4


In [210]:
entry_dataframe.to_csv(csvfile, index=False)